
# Loading binance data into trading universe

Loading CEX data into the TradingStrategy affords you the ability to work with longer historical periods with trading pairs. Using longer historical periods allows you to train your model on more data, which can lead to better results. The only downside is that you will have to wait longer for your backtests to complete. 

## Creating our assets

In the following code block, we create our ETH and USDC assets, which follow real on-chain deployment addresses. 

In [7]:
from dotenv import load_dotenv
import os

from eth_defi.chain import install_chain_middleware
from eth_defi.abi import get_deployed_contract
from web3 import Web3, HTTPProvider

from tradingstrategy.chain import ChainId
from tradeexecutor.state.identifier import AssetIdentifier

load_dotenv()

polygon_json_rpc = os.environ.get("MY_JSON_RPC_POLYGON")

web3 = Web3(HTTPProvider(polygon_json_rpc, request_kwargs={"timeout": 5}))
install_chain_middleware(web3)

# usdc with $4B supply
# https://polygonscan.com/address/0x2791bca1f2de4661ed88a30c99a7a9449aa84174
usdc_token = get_deployed_contract(
    web3, "ERC20MockDecimals.json", "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
)

# eth token.
# https://polygonscan.com//address/0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
# https://tradingstrategy.ai/trading-view/polygon/tokens/0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
eth_token = get_deployed_contract(
    web3, "ERC20MockDecimals.json", "0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619"
)


usdc_asset = AssetIdentifier(
    ChainId.polygon.value,
    usdc_token.address,
    usdc_token.functions.symbol().call(),
    usdc_token.functions.decimals().call(),
)


eth_asset = AssetIdentifier(
    ChainId.polygon.value,
    eth_token.address,
    eth_token.functions.symbol().call(),
    eth_token.functions.decimals().call(),
)




In [8]:
from tradeexecutor.state.identifier import TradingPairIdentifier

pair = TradingPairIdentifier(
    base = eth_asset,
    quote = usdc_asset,
    pool_address='0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640',
    exchange_address='0x1F98431c8aD98523631AE4a59f267346ea31F984',
    internal_id=2697765,
    fee=0.0005,
    info_url='https://tradingstrategy.ai/trading-view/ethereum/uniswap-v3/eth-usdc-fee-5'
)

## Examine dataset

In the following code block, we examine the dataset to see what it looks like. We can see that the dataset contains the following columns:

- `Date`: The date of the start of the time interval
- `open`: The price of the asset at the start of the time interval
- `high`: The highest price of the asset during the time interval
- `low`: The lowest price of the asset during the time interval
- `close`: The price of the asset at the end of the time interval
- `volume`: The volume of the asset traded during the time interval

In [11]:
import pandas as pd
from pathlib import Path

parquet_file_path = Path('../tests/binance-ETHUSDT-1h.parquet')

df = pd.read_parquet(parquet_file_path)

assert isinstance(df.index, pd.DatetimeIndex)

print(df.head())

                       open    high    low   close     volume
Date                                                         
2017-08-17 04:00:00  301.13  302.57  298.0  301.61  125.66877
2017-08-17 05:00:00  301.61  303.28  300.0  303.10  377.67246
2017-08-17 06:00:00  302.40  304.44  301.9  302.68  303.86672
2017-08-17 07:00:00  302.68  307.96  302.6  307.96  754.74510
2017-08-17 08:00:00  307.95  309.97  307.0  308.62  150.75029


In [12]:
from tradeexecutor.strategy.pandas_trader.alternative_market_data import load_candle_universe_from_parquet

universe = load_candle_universe_from_parquet(
    pair=pair,
    file=parquet_file_path,
)

Universe loaded
(<tradingstrategy.candle.GroupedCandleUniverse object at 0x7f5c253d05e0>, <tradingstrategy.candle.GroupedCandleUniverse object at 0x7f5c253d1000>)


## Using the universe in a strategy